## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import librosa
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

## 加載音頻訊號

In [ ]:
data_path = '/kaggle/input/recoding-data/UAV Recordings 1s/UAV Recordings 1s'

## 音頻訊號進行MFCCs特徵提取

In [ ]:
def load_data(data_dir):
    datasets = []
    labels = []
    categories = os.listdir(data_dir)
    for index, folder in enumerate(categories):
        for filename in os.listdir(data_dir+"/"+folder):
            data, sampling_rate = librosa.load(data_dir+"/"+folder+"/"+filename)
            mfccs_features = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=80)
            mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
            datasets.append(np.array(mfccs_scaled_features))
            labels.append(categories[index])
    return np.array(datasets), np.array(labels)

datasets, labels = load_data(data_path)

In [ ]:
datasets = datasets.reshape(datasets.shape[0],datasets.shape[1], 1, 1).astype('float32')

labelencoder = LabelEncoder()

labelsOneHot = to_categorical(labelencoder.fit_transform(labels))

## 切分成訓練資料、測試資料，8:2的比例

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datasets, labelsOneHot, test_size = 0.2, stratify = labelsOneHot, random_state = 3)

print("----------Train-------------")
print(X_train.shape)

print("----------Test--------------")
print(X_test.shape)

## 類別數量

In [ ]:
num_classes = len(np.unique(labels))

## 建立CNN模型

In [ ]:
# 建立一個Sequential模型
model = Sequential()

# 第一層卷積層，32個3x3的過濾器
model.add(Conv2D(32, kernel_size=(5, 5), padding='same',activation='relu', input_shape=(datasets.shape[1],1,1)))

# 池化層，池化窗口大小為2x2
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

# 第二層卷積層，128個3x3的過濾器
model.add(Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu'))

# 池化層，池化窗口大小為2x2
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

# 第三層卷積層，256個3x3的過濾器
model.add(Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))

# 池化層，池化窗口大小為2x2
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

# Dropout層，隨機丟棄25%的神經元
model.add(Dropout(0.25))

# Flatten層，將多維輸入一維化，常用在從卷積層到全連接層的過渡
model.add(Flatten())

# 全連接層，128個輸出單元
model.add(Dense(256, activation='relu'))

# 輸出層，有5個類別，所以輸出層的輸出維度為5
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# 選擇損失函數、優化方法及成效衡量方式
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 訓練模型

In [ ]:
train_history = model.fit(X_train, y_train, validation_split=0.2, batch_size = 100, epochs = 100, verbose=2)

## 評估訓練結果

In [ ]:
def show_train_history(train_history, train, validation):

    plt.plot(train_history.history[train])

    plt.plot(train_history.history[validation])

    plt.title('Train History')

    plt.ylabel('train')

    plt.xlabel('Epoch')

    plt.legend(['train', 'validation'], loc='upper left')

    plt.show()
    

show_train_history(train_history, 'accuracy', 'val_accuracy')
show_train_history(train_history, 'loss', 'val_loss')

## 使用未參與訓練的測試資料測試，模型準確率

In [ ]:
scores = model.evaluate(X_test,y_test,verbose=0)

print('Accuracy = ', scores[1])

## 評估訓練結果

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
confusion_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,7))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()